The goal is to get all the Retail (17000 category) venues from FourSquare Places for Greater Brisbane and remove all duplicates from the results. 
Entry points must be taken from GeoJSON file. The GeoJSON file contains a grid of points in Greater Brisbane. The task involves sending requests using FourSquare Places API for each point. The points are 6 km apart.

In [1]:
!pip install -q google-cloud google-cloud-storage google-cloud-bigquery geopy


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas as pd
import json
from datetime import datetime
import requests
from geopy.distance import geodesic
from tqdm import tqdm

fsq_authorization_key = "*" # Set `fsq_authorization_key` to authorization key
credentials_path = r"C:\Users\*\*\*.json" # Set `credentials_path` to local path to JSON file with credentials
points_table_id = "teak-surge-408913.geo.Greater Brisbane Grid 6km"
credentials = service_account.Credentials.from_service_account_file(credentials_path)
client = bigquery.Client(credentials=credentials)
export_table_id_prefix = "teak-surge-408913.geo.fsq_venues_export"
search_category = 17000

# We have the condition that the points are six kilometers apart, sitting on an imaginary grid. 
# So the distance between the nearest points that are not on the same square side is the diagonal of that square, 
# which divides it into two right triangles. The hypotenuse of these triangles (=the diagonal of the square) 
# is approximately 8485.28137. If we take a half-point search radius, we cover the entire space between the points. 
# 4243 is rounded value of the 8485.28137/2.
search_radius = 4243

Collect all point coordinates from `Greater Brisbane Grid 6km` located in the BigQuery Studio project using BigQuery's client and save them into Python dataframe.

In [3]:
get_coordinates_query = f'''SELECT geometry.coordinates[0] AS longitude, 
geometry.coordinates[1] AS latitude, 
properties.id AS point_id 
FROM `{points_table_id}`'''
point_coordinates = client.query(get_coordinates_query).to_dataframe()
print(point_coordinates.head())

    longitude   latitude  point_id
0  151.820446 -27.531805        77
1  151.820446 -27.579590        78
2  151.820446 -27.627354        79
3  151.820446 -27.675097        80
4  151.820446 -27.722819        81


Send an API request for venues in `Retail(17000)` category within search radius of `4243` from the point coordinates to the foursquare's server, remove duplicates and export obtained list of venues to a BigQuery Studio project's table.

In [4]:
# foursquare request's data
url = "https://api.foursquare.com/v3/places/search"
params = {
    "ll": "",
    "categories": f"{search_category}",
    "radius": f"{search_radius}"
}
headers = {
    "Accept": "application/json",
    "Authorization": f"{fsq_authorization_key}"
}

# Function to extract the URL from the link
def extract_next_link(link_header):
    links = link_header.split(', ')
    for link in links:
        url, rel = link.split('; ')
        if 'rel="next"' in rel:
            return url.strip('<>')

def get_fsq_page_recursive(url, params, headers):
    response = requests.request("GET", url, params=params, headers=headers)
    resp_headers = response.headers
    responce_dict = json.loads(response.text)
    venues = responce_dict["results"] #list of venues from responce
    #print(resp_headers)
    if 'Link' not in resp_headers.keys():
        return venues
    
    next_page = extract_next_link(resp_headers["Link"])
    #print(f"Next-Page: {next_page}")
    
    # we don't need to add parameters they are already in the URL
    v = get_fsq_page_recursive(next_page, [], headers)
    venues.extend(v)
    return venues

# find all venues for the points from `Greater Brisbane Grid 6km` and add it to dataframe
found_venues = []
for _, point in tqdm(point_coordinates.iterrows(), total=point_coordinates.shape[0]):
    params["ll"] = f"{point[1]},{point[0]}"
    venues = get_fsq_page_recursive(url, params=params, headers=headers)
    for fsq in venues:
        try:         
            entity = dict()
            entity["fsq_id"] = fsq["fsq_id"]
            entity["point_id"] = f"{point[2]}" 
            entity["latitude"] = fsq["geocodes"]["main"]["latitude"]
            entity["longitude"] = fsq["geocodes"]["main"]["longitude"]
            entity["address"] = fsq.get("location", {}).get("address", None) 
            entity["country"] = fsq.get("location", {}).get("country", None)
            entity["cross_street"] = fsq.get("location", {}).get("cross_street", None)
            entity["formatted_address"] = fsq.get("location", {}).get("formatted_address", None)
            entity["locality"] = fsq.get("location", {}).get("locality", None)
            entity["postcode"] = fsq.get("location", {}).get("postcode", None)
            entity["region"] = fsq.get("location", {}).get("region", None)
            entity["name"] = fsq.get("name", None)
            entity["dist"] = geodesic((point[1],point[0]), (entity["latitude"],entity["longitude"])).km
            found_venues.append(entity)
        except Exception as e:
            print(e)
            continue
            
print(f"Venues found total: {len(found_venues)}")

100%|██████████| 813/813 [11:13<00:00,  1.21it/s]

Venues found total: 6295


In [5]:
# specify schema for a table with exported data from foursquare
schema=[
    bigquery.SchemaField("fsq_id", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("point_id", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("latitude", "FLOAT64"),
    bigquery.SchemaField("longitude", "FLOAT64"),
    bigquery.SchemaField("address", "STRING"),
    bigquery.SchemaField("country", "STRING"),
    bigquery.SchemaField("cross_street", "STRING"),
    bigquery.SchemaField("formatted_address", "STRING"),
    bigquery.SchemaField("locality", "STRING"),
    bigquery.SchemaField("postcode", "STRING"),
    bigquery.SchemaField("region", "STRING"),
    bigquery.SchemaField("name", "STRING"),
    bigquery.SchemaField("dist", "FLOAT64"),
    ]

#create new table 
table_id = f'''{export_table_id_prefix}_{datetime.now().strftime("%Y%m%d%H%M%S")}'''
table = bigquery.Table(table_id, schema=schema)
table = client.create_table(table)

print(f"Created new table: {table_id}")

errors = client.insert_rows_json(table, found_venues)
if errors == []:
    print("success")
else:
    for error in errors:
        print(error)

Created new table: teak-surge-408913.geo.fsq_venues_export_20231228132038
success


In [6]:
clean_duplicates_query = f'''CREATE OR REPLACE TABLE `{table_id}` AS (
  SELECT * EXCEPT(row_num) FROM (
      SELECT *,
        ROW_NUMBER() OVER (
        PARTITION BY
        fsq_id
        ORDER BY
        dist ASC
        ) row_num
      FROM
      `{table_id}`)
  WHERE row_num=1
)'''
client.query(clean_duplicates_query)

QueryJob<project=teak-surge-408913, location=US, id=100588f9-18dc-40b2-a126-699216061e5e>